In [ ]:
#Projects/Things to know and/or look at ...
# Data Cleaning and Preparation:
#   In this case the data is relatively clean, but you may still need some preprocessing, such as scaling.
# Model Selection: 
#   Train at least two models on the dataset. Clearly indicate which metrics you used and the performance of each model. Be sure to address any imbalance in the data, as well as using an appropriate train/test data split.
# Performance Optimization: 
#   Use regularization, hyperparameter tuning, or other techniques to further optimize your chosen model and/or help select the best model.

# We need training data ... split the data into training and testing sets: X_train, X_test, y_train, y_test
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)
#
#print(X_train.shape)
#print(X_test.shape)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# import our dataset
# NOTE: the variable 'df' represents 'data frame'
#full_path = "C:\\Users\\randel.bjorkquist\\Documents\\QuickStart\\DataScience\\Student_Performance.csv"
#df = pd.read_csv(full_path, parse_dates=['Test_Date'])

#df = pd.read_csv( './data/bank-additional-full.csv'
#                ,parse_dates = ['application_date']
#                ,dtype       = { 'default': 'boolean'
#                                ,'housing': 'boolean'
#                                ,'loan': 'boolean'
#                                ,'y': 'boolean' })

df = pd.read_csv( './data/bank-additional-full.csv'
                ,dtype = { 'default': 'boolean'
                          ,'housing': 'boolean'
                          ,'loan': 'boolean'
                          ,'y': 'boolean' })

df.info()
print(df.head(5))